# Primes Visualization

Script that visualizes the factorization of integers from 1 to N as an example dataset 
to perform dimensionality reduction on.

Adapted from:
    https://gist.github.com/johnhw/dfc7b8b8519aac530ac97da226c17bd5

# Generate Prime Data

Specifically, the prime factors of integers from 1 to N encoded as binary vectors, which corresponds to an N-dimensional hyper-cube, stored in a sparse matrix. The result is then the target of dimensionality reduction algorithms.

In [ ]:
from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np
from scipy.special import expi
import scipy.sparse
from tqdm import tqdm

from primes import factorization, primesbelow, smallprimes

In [ ]:
%matplotlib ipympl

In [ ]:
prime_ix = {p:i for i,p in enumerate(smallprimes)}

Create sparse binary factor vectors for any number, and assemble into a matrix. One column for each unique prime factor. One row for each number, 0=does not have this factor, 1=does have this factor (might be repeated)

In [ ]:
def factor_vector_lil(n):
    ## approximate prime counting function (upper bound for the values we are interested in)
    ## gives us the number of rows (dimension of our space)
    d = int(np.ceil(expi(np.log(n))))    
    x = scipy.sparse.lil_matrix((n,d))
    for i in tqdm(range(2,n)): 
        for k,v in factorization(i).items():            
            x[i,prime_ix[k]] = 1
                    
    return x

# Dimensionality Reduction

In [ ]:
emb = {}

## UMAP - Universal Manifold Approximation & Projection

In [ ]:
import umap

n = 1_000_000
# n = 1000
cachepath = Path(f"umap-pts-cache-{n:.0g}.npz")
if not cachepath.is_file():
    # Generate the matrix for 1 million integers
    X = factor_vector_lil(n)

    # embed with UMAP
    emb['umap'] = umap.UMAP(metric='cosine', verbose=True, n_epochs=500).fit_transform(X)

    # save for later
    np.savez(f'pts-cache-{n:.1e}.npz', embedding=embedding)
else:
    emb['umap'] = np.load(cachepath)['embedding']

In [ ]:
import datashader as ds
import colorcet as cc
import datashader.transfer_functions as tf
import pandas as pd
import matplotlib.pyplot as plt

def render_embedding(data, **kwargs):
    size = min(data.shape[0] // 10, 100)
    canvas = ds.Canvas(500, 500, **kwargs)
    data = np.c_[data, np.arange(data.shape[0])]
    df = pd.DataFrame(data, columns=["x", "y", "i"])

    img = tf.shade(canvas.points(df, 'x', 'y'), how="eq_hist", cmap=plt.cm.viridis)
    img = tf.set_background(img, 'black')
    return img

In [ ]:
N = 50_000

In [ ]:
im = render_embedding(emb['umap'][:N])
ds.utils.export_image(im, 'umap')

## t-SNE

In [ ]:
X = factor_vector_lil(n)

In [ ]:
from sklearn import manifold

tsne = manifold.TSNE(2, perplexity=30.0, n_iter=500, init="random")
emb['tsne'] = tsne.fit_transform(X[:50_000])
emb['tsne'].shape

In [ ]:
im = render_embedding(emb['tsne'], y_range=(-51, 51), x_range=(-51, 51))
ds.utils.export_image(im, 'tsne')

## isomap

In [ ]:
iso = manifold.Isomap(n_components=2)
emb['iso'] = iso.fit_transform(X[:10_000])
emb['iso'].shape

In [ ]:
im = render_embedding(emb['iso'])
ds.utils.export_image(im, 'isomap')

In [ ]:
plt.figure(figsize=(4,4))
plt.clf()
plt.spy(X, aspect="auto", marker='.', markersize=0.1)
# plt.axis('off')
plt.ylabel("$\mathbb{Z} \in [2, 50e3]$")
plt.xlabel("Prime Factors")
plt.xticks([])
plt.yticks([])
# plt.xlim([0, 50000])
plt.show()
plt.savefig('sparsity.png')

## scratch

In [ ]:
# and save the image
fig = plt.figure(figsize=(8,8))
fig.patch.set_facecolor('black')
plt.scatter(embedding[:,0], embedding[:,1], marker='o', s=0.5, edgecolor='none',
            c=np.arange(n), cmap="magma")

plt.axis("off")
# plt.savefig(f"primes_umap_{n:.1e}_pts.png", dpi=250, facecolor='black')

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

points = hv.Points(df[:100_000], ['x', 'y'])
datashade(points, cmap=cc.fire, width=1000, height=1000)

In [ ]:
df.describe()

In [ ]:
import torch
from torch import nn

In [ ]:
lin = nn.Linear(12*12*100, 1024, False)
x = torch.randn(18, 18, 1000)
lin(x)